# Artificial Neural Network (ANN)

## Dataset

### Layout

* Columns:
	* Row Number
    * Customer ID
    * Surname
    * Credit Score
    * Geography
    * Gender
    * Age
    * Tenure
    * Balance
    * Number of Products
    * Has Credit Card
    * Is Active Member
    * Estimated Salary
    * Exited
* Rows: 1000s of observations
	* Each row represents a customer with:
	    * Identification data
	    * Demographic data
        * Bank account data
* Dataset of fictional but realistic bank customers

### Background

* One is a data scientist working for a bank
* The bank has seen unusually high churn rates, where customers are leaving the bank
* The bank wants to assess what the problem is and address it
* Six months ago, the bank first identified the problem
* The bank captured a dataset from a sample of 10,000 customers
    * This is a small fraction of their overall customer base, which numbers in the millions
    * The dataset includes everything they know about their customers
    * Based on the information captured, the bank created an estimated salary of each customer
* The bank defined a strategy where they waited six months and tracked which customers left the bank
    * The Exited column identifies whether a customer left the bank
        * 1 = Yes
        * 0 = No

### Goals

* Build a geodemographic segmentation model from an ANN using stochastic gradient descent identifying which customers are at the highest risk of leaving the bank

![Stochastic Gradient Descent](Stochastic_Gradient_Descent.png)

---

## Import Libraries

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras import Sequential

In [24]:
tf.__version__

'2.19.0'

---

## Data Preprocessing

### Import Dataset

In [25]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [26]:
print(*X[:25], sep='\n')

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
[608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
[502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]
[699 'France' 'Female' 39 1 0.0 2 0 0 93826.63]
[850 'Spain' 'Female' 43 2 125510.82 1 1 1 79084.1]
[645 'Spain' 'Male' 44 8 113755.78 2 1 0 149756.71]
[822 'France' 'Male' 50 7 0.0 2 1 1 10062.8]
[376 'Germany' 'Female' 29 4 115046.74 4 1 0 119346.88]
[501 'France' 'Male' 44 4 142051.07 2 0 1 74940.5]
[684 'France' 'Male' 27 2 134603.88 1 1 1 71725.73]
[528 'France' 'Male' 31 6 102016.72 2 0 0 80181.12]
[497 'Spain' 'Male' 24 3 0.0 2 1 0 76390.01]
[476 'France' 'Female' 34 10 0.0 2 1 0 26260.98]
[549 'France' 'Female' 25 5 0.0 2 0 0 190857.79]
[635 'Spain' 'Female' 35 7 0.0 2 1 1 65951.65]
[616 'Germany' 'Male' 45 3 143129.41 2 0 1 64327.26]
[653 'Germany' 'Male' 58 1 132602.88 1 1 0 5097.67]
[549 'Spain' 'Female' 24 9 0.0 2 1 1 14406.41]
[587 'Spain' 'Male' 45 6 0.0 1 0 0 158684.81]
[726 'France' 'Female' 24 6 0.0 2 1 1 54724.03]


In [27]:
print(*y[:25], sep='\n')

1
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0


### Encode Categorical Data

#### Label Encode Gender Column

In [28]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [29]:
print(*X[:25], sep='\n')

[619 'France' 0 42 2 0.0 1 1 1 101348.88]
[608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
[502 'France' 0 42 8 159660.8 3 1 0 113931.57]
[699 'France' 0 39 1 0.0 2 0 0 93826.63]
[850 'Spain' 0 43 2 125510.82 1 1 1 79084.1]
[645 'Spain' 1 44 8 113755.78 2 1 0 149756.71]
[822 'France' 1 50 7 0.0 2 1 1 10062.8]
[376 'Germany' 0 29 4 115046.74 4 1 0 119346.88]
[501 'France' 1 44 4 142051.07 2 0 1 74940.5]
[684 'France' 1 27 2 134603.88 1 1 1 71725.73]
[528 'France' 1 31 6 102016.72 2 0 0 80181.12]
[497 'Spain' 1 24 3 0.0 2 1 0 76390.01]
[476 'France' 0 34 10 0.0 2 1 0 26260.98]
[549 'France' 0 25 5 0.0 2 0 0 190857.79]
[635 'Spain' 0 35 7 0.0 2 1 1 65951.65]
[616 'Germany' 1 45 3 143129.41 2 0 1 64327.26]
[653 'Germany' 1 58 1 132602.88 1 1 0 5097.67]
[549 'Spain' 0 24 9 0.0 2 1 1 14406.41]
[587 'Spain' 1 45 6 0.0 1 0 0 158684.81]
[726 'France' 0 24 6 0.0 2 1 1 54724.03]
[732 'France' 1 41 8 0.0 2 1 1 170886.17]
[636 'Spain' 0 32 8 0.0 2 1 0 138555.46]
[510 'Spain' 0 38 4 0.0 1 1 0 118913.53

#### One Hot Encode Geography Column

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [31]:
print(*X[:25], sep='\n')

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
[0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]
[1.0 0.0 0.0 502 0 42 8 159660.8 3 1 0 113931.57]
[1.0 0.0 0.0 699 0 39 1 0.0 2 0 0 93826.63]
[0.0 0.0 1.0 850 0 43 2 125510.82 1 1 1 79084.1]
[0.0 0.0 1.0 645 1 44 8 113755.78 2 1 0 149756.71]
[1.0 0.0 0.0 822 1 50 7 0.0 2 1 1 10062.8]
[0.0 1.0 0.0 376 0 29 4 115046.74 4 1 0 119346.88]
[1.0 0.0 0.0 501 1 44 4 142051.07 2 0 1 74940.5]
[1.0 0.0 0.0 684 1 27 2 134603.88 1 1 1 71725.73]
[1.0 0.0 0.0 528 1 31 6 102016.72 2 0 0 80181.12]
[0.0 0.0 1.0 497 1 24 3 0.0 2 1 0 76390.01]
[1.0 0.0 0.0 476 0 34 10 0.0 2 1 0 26260.98]
[1.0 0.0 0.0 549 0 25 5 0.0 2 0 0 190857.79]
[0.0 0.0 1.0 635 0 35 7 0.0 2 1 1 65951.65]
[0.0 1.0 0.0 616 1 45 3 143129.41 2 0 1 64327.26]
[0.0 1.0 0.0 653 1 58 1 132602.88 1 1 0 5097.67]
[0.0 0.0 1.0 549 0 24 9 0.0 2 1 1 14406.41]
[0.0 0.0 1.0 587 1 45 6 0.0 1 0 0 158684.81]
[1.0 0.0 0.0 726 0 24 6 0.0 2 1 1 54724.03]
[1.0 0.0 0.0 732 1 41 8 0.0 2 1 1 170886.17]
[0.0 0.0 1.0 

### Split Dataset into Training Set and Test Set

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

**IMPORTANT!**

Feature scaling MUST be applied with deep learning when training neural networks:

* Applied across all columns
* As it is crucial that all values are normalized
* This allows the ANN to learn effectively

In [33]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [34]:
print(*X_train[:25], sep='\n')

[-1.01460667 -0.5698444   1.74309049  0.16958176 -1.09168714 -0.46460796
  0.00666099 -1.21571749  0.8095029   0.64259497 -1.03227043  1.10643166]
[-1.01460667  1.75486502 -0.57369368 -2.30455945  0.91601335  0.30102557
 -1.37744033 -0.00631193 -0.92159124  0.64259497  0.9687384  -0.74866447]
[ 0.98560362 -0.5698444  -0.57369368 -1.19119591 -1.09168714 -0.94312892
 -1.031415    0.57993469 -0.92159124  0.64259497 -1.03227043  1.48533467]
[-1.01460667 -0.5698444   1.74309049  0.03556578  0.91601335  0.10961719
  0.00666099  0.47312769 -0.92159124  0.64259497 -1.03227043  1.27652776]
[-1.01460667 -0.5698444   1.74309049  2.05611444 -1.09168714  1.73658844
  1.04473698  0.8101927   0.8095029   0.64259497  0.9687384   0.55837842]
[-1.01460667  1.75486502 -0.57369368  1.29325423 -1.09168714 -0.17749539
 -1.031415    0.44253504  0.8095029   0.64259497 -1.03227043  1.63252134]
[ 0.98560362 -0.5698444  -0.57369368  1.6128308   0.91601335  0.77954653
 -1.37744033  0.30432823 -0.92159124 -1.55619

In [35]:
print(*X_test[:25], sep='\n')

[-1.01460667  1.75486502 -0.57369368 -0.55204276 -1.09168714 -0.36890377
  1.04473698  0.8793029  -0.92159124  0.64259497  0.9687384   1.61085707]
[ 0.98560362 -0.5698444  -0.57369368 -1.31490297 -1.09168714  0.10961719
 -1.031415    0.42972196 -0.92159124  0.64259497 -1.03227043  0.49587037]
[-1.01460667 -0.5698444   1.74309049  0.57162971 -1.09168714  0.30102557
  1.04473698  0.30858264 -0.92159124  0.64259497  0.9687384  -0.42478674]
[ 0.98560362 -0.5698444  -0.57369368  1.41696129  0.91601335 -0.65601634
 -0.33936434  0.57533623 -0.92159124 -1.55619021 -1.03227043 -0.18777657]
[-1.01460667  1.75486502 -0.57369368  0.57162971  0.91601335 -0.08179119
  0.00666099  1.38961097  0.8095029   0.64259497  0.9687384   0.61684179]
[-1.01460667 -0.5698444   1.74309049  0.20050853 -1.09168714  1.73658844
 -0.68538967  1.5900207   0.8095029   0.64259497 -1.03227043 -0.01930192]
[-1.01460667 -0.5698444   1.74309049 -0.62420521  0.91601335 -0.46460796
 -1.72346566 -0.1640232   0.8095029  -1.55619

---

## Build ANN

### Initialize ANN

* The `Sequential` class is from the `models` module of the Keras libray and allows one to construct a neural network made of a sequence of layers vs. a computational graph
    * Keras is integrated into TensorFlow as of version `2.x`
* Another type of neural network is a computation graph
    * Neurons that are not connected in successive layers
    * Boltzmann machines are a type of stochastic neural network with symmetrical connections between nodes, used for unsupervised learning and modeling probability distributions
* The `ann` variable is an object instance of the `Sequential` class

In [36]:
ann = tf.keras.models.Sequential()

### Add Input layer and First Hidden Layer

* The `Dense` class is from the `layers` module of the Keras library and allows one to add a fully connected layer to an ANN
    * An object instance of this class is used to construct a connected layer
    * Parameters
        * `units` defines the number of hidden neurons in a hidden layer
        * `activation` defines the activation function
            * Rectifier activation function (`relu`) will be used for hidden layers
            * Sigmoid activation function (`sigmoid`) will be used for the output layer
* *How does one know how many neurons to create in hidden layer(s)?*
    * There is no rule of thumb
    * A satisfactory number is discovered through experimentation
        * Experiment with hyperparameters
            * These are parameters are not trained during model training
* Layers:
    * Input layer will contain `n` number of neurons, one representing each independent variable
    * Hidden layers will contain $6$ neurons, as $6$ was determined to have the best accuracy from experimentation
    * Output layer will contain $1$ neuron, representing the single dependent variable

In [37]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Add Second Hidden Layer

* The second hidden layer will simply use the same number of neurons and activation function as the first hidden layer

In [38]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Add Output Layer

* The same `add` method and `Dense` class is used to construct the output layer except it:
    * Has only $1$ neuron
        * Since there is only $1$ dependent variable
    * Has Sigmoid activation function
        * Since it gets the predictions and determines the probability of the binary outcome being $1$

In [39]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

* When making predictions for more than two categories for classification, use the `softmax` activation function
* When making predictions for regression, use the regression activation functions

---

## Train ANN

### Compile ANN

* `compile` method of `Sequential` class compiles the neural network
    * Parameters
        * `optimizer` defines the algorithm used to minimize loss
            * Calculates the gradient of the cost function
            * Updates the weights by moving in the direction of the negative gradient
            * Optimizer will eventually converge on the global minimum, which is the acceptable level of error
            * `adam` is a very performant optimizer that performs stochastic gradient descent (SGD)
        * `loss` defines the algorithm to compute the cost function value, which is the difference between predictions and actual values
            * When making binary predictions, use the `binary_crossentropy` loss function
            * When making one or two categorical predictions, use the `categorical_crossentropy` loss function
        * `metrics` defines the list of metrics to be evaluated by the model during training and testing

In [40]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train ANN on Training Set

* `fit` method of `Sequential` class trains the neural network on the training set
    * Parameters
        * `x` defines the matrix of features
        * `y` defines the dependent variable vector
        * `batch_size` defines the batch size of predictions per iteration to compare to the number of actual values
            * This is a hyperparameter
            * $32$ is a default batch size
        * `epochs` defines the number of full iterations of a dataset
            * A neural network must be trained over $n$ number of epochs to improve its accuracy over time
            * This is a hyperparameter

In [41]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.7395 - loss: 0.6012
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - accuracy: 0.7950 - loss: 0.4818
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - accuracy: 0.8057 - loss: 0.4390
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.8115 - loss: 0.4394
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.8219 - loss: 0.4200
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.8237 - loss: 0.4186
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.8326 - loss: 0.4145
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - accuracy: 0.8308 - loss: 0.4089
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - accuracy: 0.8355 - loss: 0.4016
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.8398 - loss: 0.3982
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.8371 - loss: 0.4045
Epoch 12/100
250/25

---

## Make Predictions and Evaluate Model

### Predict Result of Single Observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predict Test Set Results

### Make Confusion Matrix

### Compute Accuracy Score